In [1]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
 pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [5]:
import splitfolders

In [11]:
input_folder=r"C:\Users\madhu\Desktop\potato-disease-classification-main-main\dataset"

In [12]:
input_folder

'C:\\Users\\madhu\\Desktop\\potato-disease-classification-main-main\\dataset'

In [13]:
splitfolders.ratio(input_folder, output="output",seed=101,ratio=(.8,.1,.1),group_prefix=None)

Copying files: 2152 files [00:03, 564.51 files/s]


In [14]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'C:\Users\madhu\Desktop\potato-disease-classification-main-main\train_test_val\train'
valid_path = r'C:\Users\madhu\Desktop\potato-disease-classification-main-main\train_test_val\test'

In [17]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
vgg16= VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [18]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [22]:
  # useful for getting number of output classes
folders = glob(r'C:\Users\madhu\Desktop\potato-disease-classification-main-main\train_test_val\train\*')

In [23]:
len(folders)

3

In [24]:
folders

['C:\\Users\\madhu\\Desktop\\potato-disease-classification-main-main\\train_test_val\\train\\Potato___Early_blight',
 'C:\\Users\\madhu\\Desktop\\potato-disease-classification-main-main\\train_test_val\\train\\Potato___healthy',
 'C:\\Users\\madhu\\Desktop\\potato-disease-classification-main-main\\train_test_val\\train\\Potato___Late_blight']

In [25]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [26]:
prediction = Dense(3, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [29]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [30]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [32]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r"C:\Users\madhu\Desktop\potato-disease-classification-main-main\train_test_val\train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1721 images belonging to 3 classes.


In [36]:
test_set = test_datagen.flow_from_directory(r"C:\Users\madhu\Desktop\potato-disease-classification-main-main\train_test_val\test",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 216 images belonging to 3 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\madhu\AppData\Local\Temp\ipykernel_5956\4130870686.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/20
54/54 [==============================] - 576s 11s/step - loss: 0.7405 - accuracy: 0.7525 - val_loss: 0.2399 - val_accuracy: 0.9120
Epoch 2/20
54/54 [==============================] - 507s 9s/step - loss: 0.1987 - accuracy: 0.9309 - val_loss: 0.2553 - val_accuracy: 0.8935
Epoch 3/20
54/54 [==============================] - 512s 9s/step - loss: 0.1368 - accuracy: 0.9599 - val_loss: 0.1487 - val_accuracy: 0.9537
Epoch 4/20
54/54 [==============================] - 514s 10s/step - loss: 0.1243 - accuracy: 0.9611 - val_loss: 0.1356 - val_accuracy: 0.9583
Epoch 5/20
54/54 [==============================] - 513s 9s/step - loss: 0.1255 - accuracy: 0.9611 - val_loss: 0.2159 - val_accuracy: 0.9120
Epoch 6/20
54/54 [==============================] - 500s 9s/step - loss: 0.0894 - accuracy: 0.9756 - val_loss: 0.1341 - val_accuracy: 0.9676
Epoch 7/20
54/54 [==============================] - 497s 9s/step - loss: 0.0634 - accuracy: 0.9791 - val_loss: 0.1209 - val_accuracy: 0.9583
Epoch 8/20


In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')